In [ ]:
import argparse
import math
import os
from IPython.display import clear_output
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "../")
from stylegan_models import StyledGenerator
from utils.ptq_utils import *
from utils.brecq import *
import torch

In [ ]:
weights_path = "../weights" # path where you placed pretrained weights
tmp_data_dir = "/media/data/lsq-brecq" # path where intermediate data will be stored
generated_data_dir = "/media/data/mlproject" # path where prepare_data.py script generated data
save_path = "./lsq-brecq.pth" # where to save model?

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=["lr", "FID", "weighted", "lr_lsq"])

res_dir = tmp_data_dir
model_path = f"{weights_path}/fp_model.pth"
if not os.path.exists(res_dir):
    os.mkdir(tmp_data_dir)

lr_lsq  = 1e-5 # lr for quantization parameters
per_channel = True # per channel quantization
n_bits = 8 # bits for weights quantization
lr = 1e-2 # lr for weights and biases
weighted = False # weighted MSE ? (was used in BRECQ article)

q_params = {'q_max': 0.9999, 'q_min': 1 - 0.9999,
            'bits_w': n_bits, "per_channel":per_channel,
            'lsq_act': True, 'lsq_weight': True, "adaround":False}            


model = get_lbl_quant_model_stylegan(model_path, save_path=save_path, lr=lr, lr_lsq=lr_lsq,
                                     q_params=q_params,
                                     plot=False, weighted=weighted, iters=2000)
   
qfid = get_fid(model, tmp_data_dir, reference_data=generated_data_dir + "/fp_data",
               use_cache=True, qfid=True)

fid = get_fid(model, tmp_data_dir, reference_data=generated_data_dir + "/real_images",
               use_cache=True, qfid=False)